In [ ]:
import pandas as pd
import numpy as np

In [ ]:
grand_df = pd.read_csv("training_data.csv")

In [ ]:
X = np.array(grand_df[0].tolist())
# convert to reflectance value
X = X*0.0001

In [ ]:
crop_map = {"OTHER": 0, "SPRING": 1, "RATOON": 2, 'BARREN': 3, 'BUILTUP': 4, 'WATER': 5, 'PERENNIAL': 6, 'UC': 7}
crop_map

In [ ]:
y = grand_df[1].map(crop_map)

In [ ]:
import tensorflow
from tensorflow.keras.utils import to_categorical

Y = to_categorical(y, num_classes=8)

### nodel part

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import (
    Input, LSTM, Bidirectional, Dense,
    BatchNormalization, Dropout, Concatenate
)
from tensorflow.keras.models import Model


In [ ]:
timesteps = 23
optical_features = 4
num_classes = 8


In [ ]:
optical_input = Input(shape=(timesteps, optical_features), name="optical_input")

x_opt = Bidirectional(
    LSTM(
        100,
        return_sequences=True,
        dropout=0.0,
        recurrent_dropout=0.2
    )
)(optical_input)

x_opt = Bidirectional(
    LSTM(
        100,
        return_sequences=False,
        dropout=0.0,
        recurrent_dropout=0.2
    )
)(x_opt)

x_opt = BatchNormalization()(x_opt)
fc = Dense(100, activation="relu")(x_opt)
fc = BatchNormalization()(fc)
fc = Dropout(0.5)(fc)
output = Dense(num_classes, activation="softmax", name="output")(fc)


In [ ]:
model = Model(
    inputs=optical_input,
    outputs=output
)

model.summary()


In [ ]:
model.compile(
    optimizer="adam",
    loss="categorical_crossentropy",
    metrics=["accuracy"]
)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X,
    Y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

In [ ]:
history = model.fit(
    X_train,
    y_train,
    validation_data=(X_val, y_val),
    epochs=80,
    batch_size=32,
    shuffle=True
)

In [ ]:
model.save("Deep_LSTM.keras")